In [27]:
import numpy as np
import os

from keras.utils import to_categorical
from pommerman.agents import SimpleAgent
from pommerman.configs import ffa_competition_env
from pommerman.envs.v0 import Pomme

In [28]:
initial_rollouts = 1200
train_data_path = './dagger/train_data/'
train_data_obs = 'obs.npy'
train_data_labels = 'labels.npy'
train_data_reward = 'reward.npy'
train_data_done = 'done.npy'
if not os.path.isdir(train_data_path):
    os.makedirs(train_data_path)

In [29]:
# Environment wrapper
class Stimulator:
    def __init__(self, env, config):
        self.env = env
        self.init(config)
        self.episode_number = 0

    def init(self, config):
        self.env.seed(0)
        # Add 3 random agents
        agents = []
        for agent_id in range(4):
            agents.append(SimpleAgent(config["agent"](agent_id, config["game_type"])))
        self.env.set_agents(agents)
        self.env.set_init_game_state(None)

    def stimulate(self, num_rollouts, render=False, logging=False):
        observations = []
        actions = []
        dones = []
        rewards = []

        for i in range(num_rollouts):
            self.episode_number += 1
            obs = self.env.reset()
            done = False
            episode_steps = 0
            
            try:
                episode_observations = []
                episode_actions = []
                episode_dones = []
                episode_rewards = []
                while not done:
                    if render:
                        self.env.render()
                    all_actions = self.env.act(obs)
                    obs, reward, done, _ = self.env.step(all_actions)
                    episode_steps += 1

                    episode_observations.append(obs)
                    episode_actions.append(all_actions)
                    episode_rewards.append(reward)
                    episode_dones.append(done)
            except:
                print("Error occurs")
                i -= 1
                continue
            
            observations.extend(episode_observations)
            actions.extend(episode_actions)
            dones.extend(episode_dones)
            rewards.extend(episode_rewards)
            print('rollout %i/%i' % (i + 1, num_rollouts))
        return np.array(observations), np.array(to_categorical(actions, self.env.action_space.n)), np.array(rewards), np.array(dones)


In [30]:
# Instantiate the environment
config = ffa_competition_env()
env = Pomme(**config["env_kwargs"])

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [ ]:
# Generate training data
stimulator = Stimulator(env, config)
observations, actions, rewards, dones = stimulator.stimulate(num_rollouts=initial_rollouts)

rollout 1/1200
rollout 2/1200
rollout 3/1200
rollout 4/1200
rollout 5/1200
rollout 6/1200
rollout 7/1200
rollout 8/1200
rollout 9/1200
rollout 10/1200
rollout 11/1200
rollout 12/1200
rollout 13/1200
rollout 14/1200
rollout 15/1200
rollout 16/1200
rollout 17/1200
rollout 18/1200
rollout 19/1200
rollout 20/1200
rollout 21/1200
rollout 22/1200
rollout 23/1200
rollout 24/1200
rollout 25/1200
rollout 26/1200
rollout 27/1200
rollout 28/1200
rollout 29/1200
rollout 30/1200
rollout 31/1200
rollout 32/1200
rollout 33/1200
rollout 34/1200
rollout 35/1200
rollout 36/1200
rollout 37/1200
rollout 38/1200
rollout 39/1200
rollout 40/1200
rollout 41/1200
rollout 42/1200
rollout 43/1200
rollout 44/1200
rollout 45/1200
rollout 46/1200
rollout 47/1200
rollout 48/1200
rollout 49/1200
rollout 50/1200
rollout 51/1200
rollout 52/1200
rollout 53/1200
rollout 54/1200
rollout 55/1200
rollout 56/1200
rollout 57/1200
rollout 58/1200
rollout 59/1200
rollout 60/1200
rollout 61/1200
rollout 62/1200
rollout 63/1200
r

In [ ]:
np.save(train_data_path + train_data_obs, observations)
np.save(train_data_path + train_data_labels, actions)
np.save(train_data_path + train_data_reward, rewards)
np.save(train_data_path + train_data_done, dones)

#print(np.sum(training_data_labels, axis=0) / np.sum(training_data_labels))

In [ ]:
observations.shape, actions.shape, rewards.shape, dones.shape